In [1]:
import json
import pandas as pd
import os
import librosa
import random
import IPython.display as ipd

## Vocab Dict

In [2]:
df_phon_del = pd.read_json('dfs/phondel/Intervention_PhonDel_df.json')
df_decoding = pd.read_json('dfs/deco/Intervention_Decoding_df.json')
print(df_decoding.columns )
print(df_phon_del.columns)
# concatenate the two dataframes
print(df_decoding.columns == df_phon_del.columns)
df = pd.concat([df_decoding, df_phon_del], ignore_index=True)

df = df_decoding
df

Index(['participant_id', 'session_id', 'language', 'session_form', 'phase',
       'config_id', 'filename', 'reference_text', 'words_human_transcription',
       'human_transcription', 'accuracy', 'accuracy_og', 'notes', 'comments',
       'human_alignment', 'asr_response', 'asr_comparison', 'audio_length',
       'process_time', 'filepath'],
      dtype='object')
Index(['participant_id', 'session_id', 'language', 'session_form', 'phase',
       'config_id', 'filename', 'reference_text', 'words_human_transcription',
       'human_transcription', 'accuracy', 'accuracy_og', 'notes', 'comments',
       'human_alignment', 'asr_response', 'asr_comparison', 'audio_length',
       'process_time', 'filepath'],
      dtype='object')
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True]


,participant_id,session_id,language,session_form,phase,config_id,filename,reference_text,words_human_transcription,human_transcription,accuracy,accuracy_og,notes,comments,human_alignment,asr_response,asr_comparison,audio_length,process_time,filepath
0,3101_edugame2023,5b0b21bc5c9d4c0a95340ed6e97b0fc1,fr,B,PSEUDO,config_B_pseudo_1,3101_edugame2023_9a4621529c23405c8d2681287fed3...,jol puli ruit paroce reux lamon bente nardé du...,jol pui puli ruit paroce reux lamon binte nar...,jol pui puli rui pa.rosse re lamon bin.te nar ...,2 2 2 1 2 2 0 1 2 1 2 2,2 2 2 1 2 2 0 1 2 1 2 2,self_correction sounding_out,None,,,,27.387937,NaT,../Intervention_T1_data/recordings/deco_trials...
1,3101_edugame2023,5b0b21bc5c9d4c0a95340ed6e97b0fc1,fr,B,PSEUDO,config_B_pseudo_2,3101_edugame2023_9a4621529c23405c8d2681287fed3...,digacelle pireau lisoie milvenet diège conjeau...,dik di dijacelle pireau lisoi milvainé diège ...,dik di dijacelle piro li.soi milvain.né diège ...,0 2 0 0 2 0 0 0 0 2 0 2,0 2 0 0 2 0 0 0 0 2 0 2,self_correction sounding_out,None,,,,37.668571,NaT,../Intervention_T1_data/recordings/deco_trials...
2,3101_edugame2023,5b0b21bc5c9d4c0a95340ed6e97b0fc1,fr,B,WORD,config_B_easy_2,3101_edugame2023_c38eb5ccc4bb4fac8add25685c799...,haie brume chambre demande style crâne fureur ...,air brume chambre demande style cra crâne u ...,air brume chambre demande style {cra.}crâne u ...,0 2 2 2 2 1 1 1 2 1 2 0,0 2 2 2 2 1 1 1 2 1 2 0,sounding_out,None,,,,30.476190,NaT,../Intervention_T1_data/recordings/deco_trials...
3,3101_edugame2023,5b0b21bc5c9d4c0a95340ed6e97b0fc1,fr,B,WORD,config_B_complex_1,3101_edugame2023_c38eb5ccc4bb4fac8add25685c799...,siège coeur basket récit sept clef gourdin tra...,siège pair basket récite sept cleffe cou gour...,siège pair basket récite sept cleffe cou gourg...,2 0 2 0 2 0 0 2 2 2 1 1,2 0 2 0 2 0 0 2 2 2 1 1,sounding_out,None,,,,32.020317,NaT,../Intervention_T1_data/recordings/deco_trials...
4,3101_edugame2023,5b0b21bc5c9d4c0a95340ed6e97b0fc1,fr,B,WORD,config_B_complex_2,3101_edugame2023_c38eb5ccc4bb4fac8add25685c799...,écho logis intelligence opposition creux foire...,écho logis intelligence opposition creux foir...,écho logie inte.ligence o.pposi.tion creux foi...,0 2 1 1 2 2 2 0 1 0 1 1,0 2 1 1 2 2 2 0 1 0 1 1,sounding_out,None,,,,37.221587,NaT,../Intervention_T1_data/recordings/deco_trials...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,4006_edugame2023,None,fr,B,WORD,config_B_easy_2,4006_edugame2023_68a1be2a0a1d43f4b0463dc99ee73...,haie brume chambre demande style crâne fureur ...,haie breme chambe demande style cane fur fure...,é breme chambe demande style cane fur fureur m...,2 0 0 2 2 0 2 1 2 1 1 2,2 0 0 2 2 0 2 1 2 1 1 2,sounding_out self_correction,None,,,,21.455238,NaT,../Intervention_T1_data/recordings/deco_trials...
966,4006_edugame2023,None,fr,B,WORD,config_B_complex_1,4006_edugame2023_68a1be2a0a1d43f4b0463dc99ee73...,siège coeur basket récit sept clef gourdin tra...,siège coeur baskette récit sept scléffe goudi...,siège keur baskette ré.ci sette scléffe gou.di...,2 2 2 1 2 0 0 2 2 2 1 1,2 2 2 1 2 0 0 2 2 2 1 1,sounding_out,None,,,,34.377143,NaT,../Intervention_T1_data/recordings/deco_trials...
967,4006_edugame2023,None,fr,B,WORD,config_B_complex_2,4006_edugame2023_68a1be2a0a1d43f4b0463dc99ee73...,écho logis intelligence opposition creux foire...,écho logis intéligen opozizion creux foire ch...,é.cho lo.gie in.té.ligen o.pozizion creu foire...,0 1 0 0 2 2 2 0 0 0 2 0,0 1 0 0 2 2 2 0 0 0 2 0,sounding_out,None,,,,23.283810,NaT,../Intervention_T1_data/recordings/deco_trials...
968,4006_edugame2023,None,fr,B,PSEUDO,config_B_pseudo_1,4006_edugame2023_97a6892868d5420687fa49368f83b...,jol puli ruit paroce reux lamon bente nardé du...,jol pulpuli ruit paroce reux lamon bente nard...,jol pul.puli r.rui pa.rosse reu lamon bente na...,2 1 1 1 2 2 2 1 2 1 0 0,2 1 1 1 2 2 2 1 2 1 0 1,sounding_out,None,,,,24.665397,NaT,../Intervention_T1_data/recordings/deco_trials...


In [3]:
def extract_all_chars(df):
  all_text = " ".join(df["words_human_transcription"])
  return list(set(all_text))

# for all the dataframes in the lists extract the characters in the column "words_human_transcription" and store in a list
vocab_list = extract_all_chars(df)
# flatten the list of characters
vocab_list = list(set([item for sublist in vocab_list for item in sublist]))
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 '(': 1,
 '_': 2,
 'a': 3,
 'b': 4,
 'c': 5,
 'd': 6,
 'e': 7,
 'f': 8,
 'g': 9,
 'h': 10,
 'i': 11,
 'j': 12,
 'k': 13,
 'l': 14,
 'm': 15,
 'n': 16,
 'o': 17,
 'p': 18,
 'q': 19,
 'r': 20,
 's': 21,
 't': 22,
 'u': 23,
 'v': 24,
 'x': 25,
 'y': 26,
 'z': 27,
 '\xa0': 28,
 'â': 29,
 'ç': 30,
 'è': 31,
 'é': 32,
 'ê': 33,
 'ô': 34,
 'û': 35}

In [4]:
odd_chars = ['\xa0', '-', '(']
# Loop through the DataFrame to find and replace odd characters
for index, row in df.iterrows():
    for char in odd_chars:
        if char in row["words_human_transcription"]:
            print(f"{row['words_human_transcription']} ({index})")
            updated_value = row["words_human_transcription"].replace(char, '')
            df.loc[index, "words_human_transcription"] = updated_value

 (adult_support    (462)
 piège femme  finit mille mille  ke sér jadin précision  lieu million débar  (549)


there is a row where transcription without annotations is  (adult_support which was not removed due to a mistake in the format, so we remove it here

In [5]:
# replace words_human_transcription of row 462 with empty string if it contains the word adult
if 'adult' in df.loc[462, "words_human_transcription"]:
    df.loc[462, "words_human_transcription"] = ''

For the training part, since each token of the model is a character, I decided to match some of them to others as given by the following dictionnary: 

In [6]:
char_dict = {
    'â': 'a',
    'œ': 'oe',
    'û': 'u',
    'ô': 'o',
}

def replace_special_characters(transcription):
    for k, v in char_dict.items():
        if k in transcription:
            print(f"Replacing {k} by {v}", transcription)
            transcription = transcription.replace(k, v)
            print(f"New transcription: {transcription}")
    return transcription
df['words_human_transcription'] = df['words_human_transcription'].apply(replace_special_characters)

Replacing â by a  air brume chambre demande style  cra crâne u fure fureur manivelle hôtel cordon chanson pompier  
New transcription:  air brume chambre demande style  cra crane u fure fureur manivelle hôtel cordon chanson pompier  
Replacing ô by o  air brume chambre demande style  cra crane u fure fureur manivelle hôtel cordon chanson pompier  
New transcription:  air brume chambre demande style  cra crane u fure fureur manivelle hotel cordon chanson pompier  
Replacing â by a  lait désir joie escalope débéret vingt boulot bouleau bou bonheur pâté voix façade argile  
New transcription:  lait désir joie escalope débéret vingt boulot bouleau bou bonheur paté voix façade argile  
Replacing û by u  liède soeur redré fusil fil août garçon produitssion gaz dieux tension déménagement 
New transcription:  liède soeur redré fusil fil aout garçon produitssion gaz dieux tension déménagement 
Replacing ô by o  et brumeu sembreu demande si situsitile cra cran freu mélivelle ol ote hôtel crodon 

In [7]:
# for all the dataframes in the lists extract the characters in the column "words_human_transcription" and store in a list
vocab_list = extract_all_chars(df)
vocab_list = list(set([item for sublist in vocab_list for item in sublist]))
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}

# Insert 'w' with value just after 'v' and shift all other values by 1
vocab_dict = {k: v for k, v in vocab_dict.items()}

# Shift all values after 'w' by 1
for key in list(vocab_dict.keys()):
    if vocab_dict[key] > vocab_dict['v']:
        vocab_dict[key] += 1
vocab_dict['w'] = vocab_dict['v'] + 1


# Remove entries in vocab_dict__ that are in char_dict
vocab_dict = {k: v for k, v in vocab_dict.items() if k not in char_dict}

# Remove entries in vocab_dict__ that are in odd_chars
vocab_dict = {k: v for k, v in vocab_dict.items() if k not in odd_chars}

# Sort the dictionary by values
vocab_dict = {k: v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

# Replace the space key with '|'
if " " in vocab_dict:
    vocab_dict["|"] = vocab_dict[" "]
    del vocab_dict[" "]

# Add special tokens
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
"""
# Save the vocabulary dictionary to a JSON file
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)
"""
vocab_dict

{'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 'ç': 27,
 'è': 28,
 'é': 29,
 'ê': 30,
 '|': 0,
 '[UNK]': 31,
 '[PAD]': 32}

In [8]:
rand_index = random.randint(0, len(df) - 1)
reference_text = df.loc[rand_index, 'reference_text']
transcript = df.loc[rand_index, 'words_human_transcription']
annotated = df.loc[rand_index, 'human_transcription']
accuracy = df.loc[rand_index, 'accuracy']
audio_filename = df.loc[rand_index, 'filename']
audio_path = df.loc[rand_index, 'filepath']

# Load audio data from the selected path using librosa
audio_data, sampling_rate = librosa.load(audio_path, sr=16000)

# Print information
print("Random Integer:", rand_index)
print("Audio Filename:", audio_filename)
print("Reference Text:", reference_text)
print("Transcript:", transcript)
print("Annotated:", annotated)
print("Accuracy:", accuracy)
print("Input Array Shape:", audio_data.shape)
print("Sampling Rate:", sampling_rate)

# Display audio using IPython.display.Audio
ipd.Audio(data=audio_data, autoplay=True, rate=sampling_rate)

Random Integer: 760
Audio Filename: 3745_edugame2023_66c6a1392d494c5ba27b6f6fb66214a9_0b8f78667dcb403485f588f2efb457e5.wav
Reference Text: digacelle pireau lisoie milvenet diège conjeau beuil brumidurence héteur tive pléfation sorpile
Transcript:  digacelle pireau  loiz lisoie milvenet diège canjo beèl brumidurence héteur tive pléfation sorpile 
Annotated: digacelle piro {loiz.}lizoi mil.ven.né diège can.jo beèl brumidurence éteur tive plé.fa.ssion s.orpil
Accuracy: 2 2 1 0 2 0 0 2 2 2 1 1
Input Array Shape: (321536,)
Sampling Rate: 16000


## Data Split

In [9]:
df

,participant_id,session_id,language,session_form,phase,config_id,filename,reference_text,words_human_transcription,human_transcription,accuracy,accuracy_og,notes,comments,human_alignment,asr_response,asr_comparison,audio_length,process_time,filepath
0,3101_edugame2023,5b0b21bc5c9d4c0a95340ed6e97b0fc1,fr,B,PSEUDO,config_B_pseudo_1,3101_edugame2023_9a4621529c23405c8d2681287fed3...,jol puli ruit paroce reux lamon bente nardé du...,jol pui puli ruit paroce reux lamon binte nar...,jol pui puli rui pa.rosse re lamon bin.te nar ...,2 2 2 1 2 2 0 1 2 1 2 2,2 2 2 1 2 2 0 1 2 1 2 2,self_correction sounding_out,None,,,,27.387937,NaT,../Intervention_T1_data/recordings/deco_trials...
1,3101_edugame2023,5b0b21bc5c9d4c0a95340ed6e97b0fc1,fr,B,PSEUDO,config_B_pseudo_2,3101_edugame2023_9a4621529c23405c8d2681287fed3...,digacelle pireau lisoie milvenet diège conjeau...,dik di dijacelle pireau lisoi milvainé diège ...,dik di dijacelle piro li.soi milvain.né diège ...,0 2 0 0 2 0 0 0 0 2 0 2,0 2 0 0 2 0 0 0 0 2 0 2,self_correction sounding_out,None,,,,37.668571,NaT,../Intervention_T1_data/recordings/deco_trials...
2,3101_edugame2023,5b0b21bc5c9d4c0a95340ed6e97b0fc1,fr,B,WORD,config_B_easy_2,3101_edugame2023_c38eb5ccc4bb4fac8add25685c799...,haie brume chambre demande style crâne fureur ...,air brume chambre demande style cra crane u ...,air brume chambre demande style {cra.}crâne u ...,0 2 2 2 2 1 1 1 2 1 2 0,0 2 2 2 2 1 1 1 2 1 2 0,sounding_out,None,,,,30.476190,NaT,../Intervention_T1_data/recordings/deco_trials...
3,3101_edugame2023,5b0b21bc5c9d4c0a95340ed6e97b0fc1,fr,B,WORD,config_B_complex_1,3101_edugame2023_c38eb5ccc4bb4fac8add25685c799...,siège coeur basket récit sept clef gourdin tra...,siège pair basket récite sept cleffe cou gour...,siège pair basket récite sept cleffe cou gourg...,2 0 2 0 2 0 0 2 2 2 1 1,2 0 2 0 2 0 0 2 2 2 1 1,sounding_out,None,,,,32.020317,NaT,../Intervention_T1_data/recordings/deco_trials...
4,3101_edugame2023,5b0b21bc5c9d4c0a95340ed6e97b0fc1,fr,B,WORD,config_B_complex_2,3101_edugame2023_c38eb5ccc4bb4fac8add25685c799...,écho logis intelligence opposition creux foire...,écho logis intelligence opposition creux foir...,écho logie inte.ligence o.pposi.tion creux foi...,0 2 1 1 2 2 2 0 1 0 1 1,0 2 1 1 2 2 2 0 1 0 1 1,sounding_out,None,,,,37.221587,NaT,../Intervention_T1_data/recordings/deco_trials...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,4006_edugame2023,None,fr,B,WORD,config_B_easy_2,4006_edugame2023_68a1be2a0a1d43f4b0463dc99ee73...,haie brume chambre demande style crâne fureur ...,haie breme chambe demande style cane fur fure...,é breme chambe demande style cane fur fureur m...,2 0 0 2 2 0 2 1 2 1 1 2,2 0 0 2 2 0 2 1 2 1 1 2,sounding_out self_correction,None,,,,21.455238,NaT,../Intervention_T1_data/recordings/deco_trials...
966,4006_edugame2023,None,fr,B,WORD,config_B_complex_1,4006_edugame2023_68a1be2a0a1d43f4b0463dc99ee73...,siège coeur basket récit sept clef gourdin tra...,siège coeur baskette récit sept scléffe goudi...,siège keur baskette ré.ci sette scléffe gou.di...,2 2 2 1 2 0 0 2 2 2 1 1,2 2 2 1 2 0 0 2 2 2 1 1,sounding_out,None,,,,34.377143,NaT,../Intervention_T1_data/recordings/deco_trials...
967,4006_edugame2023,None,fr,B,WORD,config_B_complex_2,4006_edugame2023_68a1be2a0a1d43f4b0463dc99ee73...,écho logis intelligence opposition creux foire...,écho logis intéligen opozizion creux foire ch...,é.cho lo.gie in.té.ligen o.pozizion creu foire...,0 1 0 0 2 2 2 0 0 0 2 0,0 1 0 0 2 2 2 0 0 0 2 0,sounding_out,None,,,,23.283810,NaT,../Intervention_T1_data/recordings/deco_trials...
968,4006_edugame2023,None,fr,B,PSEUDO,config_B_pseudo_1,4006_edugame2023_97a6892868d5420687fa49368f83b...,jol puli ruit paroce reux lamon bente nardé du...,jol pulpuli ruit paroce reux lamon bente nard...,jol pul.puli r.rui pa.rosse reu lamon bente na...,2 1 1 1 2 2 2 1 2 1 0 0,2 1 1 1 2 2 2 1 2 1 0 1,sounding_out,None,,,,24.665397,NaT,../Intervention_T1_data/recordings/deco_trials...


In [10]:
grouped_df_session = df.groupby(['language', 'session_form'])
df_list = [[] for _ in range(5)]

# Iterate over each group
for group, data in grouped_df_session:
    # Get list of participants in the group
    participants = data['participant_id'].unique()
    # Calculate the average number of '1' and '2' per 'config_id' for each participant
    # Give the words with accuracy '1' a weight of 1 and the words with accuracy '2' a weight of 2
    participant_sums_average = {}
    for participant in participants:
        participant_data = data[data['participant_id'] == participant]
        config_counts = participant_data.groupby('config_id')['accuracy'].apply(
            lambda x: (x.astype(str).str.count('2').sum()) if x.name.startswith('phondel') 
            else  x.str.count('1').sum() + x.str.count('2').sum())
        participant_sums_average[participant] = config_counts.mean()
    
    # Sort participants based on the calculated averages
    participants = sorted(participants, key=lambda x: participant_sums_average[x], reverse=True)
    # split the participants into 5 groups where participants[0] is in group 1, participants[1] is in group 2, etc.
    split_participants = [[] for _ in range(5)]
    # Distribute participants into 5 groups
    for idx, participant in enumerate(participants):
        split_participants[idx % 5].append(participant)

    # for each group of participants, get the data and append it to the corresponding list
    for i, participants in enumerate(split_participants):
        group_data = data[data['participant_id'].isin(participants)]
        df_list[i].append(group_data)


# Concatenate the dataframes in each sublist to create the final dataframes
df_1 = pd.concat(df_list[0], ignore_index=True)
df_2 = pd.concat(df_list[1], ignore_index=True)
df_3 = pd.concat(df_list[2], ignore_index=True)
df_4 = pd.concat(df_list[3], ignore_index=True)
df_5 = pd.concat(df_list[4], ignore_index=True)

# Save df1 to json to have the 20% of the data that were not used for the LM
df_1.to_json('dfs/deco/Intervention_df_1_deco.json')
df_2.to_json('dfs/deco/Intervention_df_2_deco.json')
df_3.to_json('dfs/deco/Intervention_df_3_deco.json')
df_4.to_json('dfs/deco/Intervention_df_4_deco.json')
df_5.to_json('dfs/deco/Intervention_df_5_deco.json')
df.to_json('dfs/deco/Intervention_df_cleaned_deco.json')
    
len(df_1), len(df_2), len(df_3), len(df_4), len(df_5)

(192, 192, 186, 180, 180)

In [11]:
# Store participant_id sets for each dataframe
participant_id_sets = [
    set(df_1['participant_id']),
    set(df_2['participant_id']),
    set(df_3['participant_id']),
    set(df_4['participant_id']),
    set(df_5['participant_id'])
]
# Check intersections between each pair of sets
all_intersections_empty = True
for i in range(len(participant_id_sets)):
    for j in range(i + 1, len(participant_id_sets)):
        intersection_size = len(participant_id_sets[i].intersection(participant_id_sets[j]))
        if intersection_size > 0:
            all_intersections_empty = False
print("No participant is present in more than one dataframe:", all_intersections_empty)


No participant is present in more than one dataframe: True


In [12]:
accuracy_result_df = pd.DataFrame(df_1[['language', 'config_id']].drop_duplicates())

# Process each DataFrame and add the results to the result DataFrame
for i, dfi in enumerate([df_1, df_2, df_3, df_4, df_5], start=1):
    accuracy_counts = dfi.groupby(['language', 'config_id'])['accuracy'].apply(
            lambda x: (x.astype(str).str.count('2').sum()) if x.name[1].startswith('phondel') 
            else  x.str.count('1').sum() + x.str.count('2').sum())
    accuracy_result_df = accuracy_result_df.merge(accuracy_counts.reset_index().rename(columns={'accuracy': f'df{i}'}), on=['language', 'config_id'], how='left')

# Add the average accuracy per (language, config_id) for the whole dataset
average_accuracy = df.groupby(['language', 'config_id'])['accuracy'].apply(
            lambda x: (x.astype(str).str.count('2').sum() / 5) if x.name[1].startswith('phondel') 
            else  (x.str.count('1').sum() + x.str.count('2').sum()) / 5)
accuracy_result_df = accuracy_result_df.merge(average_accuracy.reset_index().rename(columns={'accuracy': 'avg_df'}), on=['language', 'config_id'], how='left')

# Add the accuracy per (language, config_id) for the whole dataset
average_accuracy = df.groupby(['language', 'config_id'])['accuracy'].apply(
            lambda x: (x.astype(str).str.count('2').sum()) if x.name[1].startswith('phondel') 
            else  x.str.count('1').sum() + x.str.count('2').sum()
)
accuracy_result_df = accuracy_result_df.merge(average_accuracy.reset_index().rename(columns={'accuracy': 'df'}), on=['language', 'config_id'], how='left')

print('Accuracy scores across the 5 sub-dataframes and average accuracy score for the whole dataframe:')
accuracy_result_df = accuracy_result_df.sort_values(['config_id'])
print(accuracy_result_df)
print(accuracy_result_df[['config_id', 'df']])


Accuracy scores across the 5 sub-dataframes and average accuracy score for the whole dataframe:
   language           config_id  df1  df2  df3  df4  df5  avg_df   df
3        fr  config_A_complex_1   71   80   68   68   65    70.4  352
0        fr  config_A_complex_2   73   60   59   57   55    60.8  304
1        fr     config_A_easy_1   81   87   87   79   67    80.2  401
2        fr     config_A_easy_2   83   86   77   78   70    78.8  394
4        fr   config_A_pseudo_1   75   69   76   74   70    72.8  364
5        fr   config_A_pseudo_2   64   54   57   54   62    58.2  291
9        fr  config_B_complex_1   67   83   79   76   72    75.4  377
10       fr  config_B_complex_2   74   59   60   59   64    63.2  316
11       fr     config_B_easy_1  100   95   89   91   91    93.2  466
8        fr     config_B_easy_2   80   90   86   92   86    86.8  434
6        fr   config_B_pseudo_1   87   75   86   77   77    80.4  402
7        fr   config_B_pseudo_2   79   68   60   54   51    62.4

In [13]:
# show in df1 all rows with config_id 'phondel_A_pseudo_0'
df_1[df_1['config_id'] == 'phondel_A_pseudo_0']

,participant_id,session_id,language,session_form,phase,config_id,filename,reference_text,words_human_transcription,human_transcription,accuracy,accuracy_og,notes,comments,human_alignment,asr_response,asr_comparison,audio_length,process_time,filepath


In [13]:
audio_result_df = pd.DataFrame(df_1[['language', 'config_id']].drop_duplicates())

# Process each DataFrame and add the results to the result DataFrame
for i, dfi in enumerate([df_1, df_2, df_3, df_4, df_5], start=1):
    # Group by 'language' and 'config_id', summing 'audio_length', and reset index
    audio_sum = dfi.groupby(['language', 'config_id'])['audio_length'].sum().reset_index()
    audio_sum.rename(columns={'audio_length': f'df{i}'}, inplace=True)
    audio_result_df = pd.merge(audio_result_df, audio_sum, on=['language', 'config_id'], how='left')

# Calculate the average audio_length per (language, config_id) for the whole dataset
average_audio_length = df.groupby(['language', 'config_id'])['audio_length'].sum().div(5).reset_index()
average_audio_length['audio_length'] = average_audio_length['audio_length'].round(2)
average_audio_length.rename(columns={'audio_length': 'avg_df'}, inplace=True)
audio_result_df = pd.merge(audio_result_df, average_audio_length, on=['language', 'config_id'], how='left')

# Calculate the audio_length per (language, config_id) for the whole dataset
average_audio_length = df.groupby(['language', 'config_id'])['audio_length'].sum().reset_index()
average_audio_length['audio_length'] = average_audio_length['audio_length'].round(2)
average_audio_length.rename(columns={'audio_length': 'df'}, inplace=True)
audio_result_df = pd.merge(audio_result_df, average_audio_length, on=['language', 'config_id'], how='left')

# Round the audio lengths to the nearest integer
audio_result_df.iloc[:, 2:] = audio_result_df.iloc[:, 2:].round(decimals=0).astype(int)

# Print the resulting DataFrame
print("Total audio length:", audio_result_df['df'].sum())
print("\nTotal audio length per config_id across all dataframes and average audio length:")

# sort dataframe entries by config_id 
audio_result_df = audio_result_df.sort_values(by=['config_id'])
print(audio_result_df)

print(audio_result_df[['config_id', 'df']])


Total audio length: 24218.0

Total audio length per config_id across all dataframes and average audio length:
   language           config_id    df1    df2    df3    df4    df5  avg_df  \
3        fr  config_A_complex_1  208.0  210.0  149.0  251.0  198.0   203.0   
0        fr  config_A_complex_2  298.0  259.0  208.0  301.0  243.0   262.0   
1        fr     config_A_easy_1  207.0  243.0  177.0  241.0  177.0   209.0   
2        fr     config_A_easy_2  211.0  243.0  197.0  255.0  195.0   220.0   
4        fr   config_A_pseudo_1  239.0  218.0  177.0  258.0  164.0   211.0   
5        fr   config_A_pseudo_2  282.0  278.0  222.0  273.0  256.0   262.0   
9        fr  config_B_complex_1  263.0  274.0  360.0  305.0  335.0   307.0   
10       fr  config_B_complex_2  329.0  253.0  363.0  331.0  373.0   330.0   
11       fr     config_B_easy_1  268.0  229.0  344.0  314.0  324.0   296.0   
8        fr     config_B_easy_2  254.0  230.0  375.0  303.0  338.0   300.0   
6        fr   config_B_pseudo_1 

In [15]:
count_result_df = pd.DataFrame(df_1[['language', 'config_id']].drop_duplicates())

# Process each DataFrame and add the counts to the result DataFrame
for i, dfi in enumerate([df_1, df_2, df_3, df_4, df_5], start=1):
    counts = dfi.groupby(['language', 'config_id']).size().reset_index(name=f'df{i}')
    count_result_df = count_result_df.merge(counts, on=['language', 'config_id'], how='left')

# Add the average count per (language, config_id) for the whole dataset
average_counts = df.groupby(['language', 'config_id']).size().div(5).reset_index(name='avg_df')
count_result_df = count_result_df.merge(average_counts, on=['language', 'config_id'], how='left')

# Add the  count per (language, config_id) for the whole dataset
average_counts = df.groupby(['language', 'config_id']).size().reset_index(name='df')
count_result_df = count_result_df.merge(average_counts, on=['language', 'config_id'], how='left')

print("Total count:", count_result_df['df'].sum())
print("\nCount per config_id for the 5 sub-dataframes and average count for the whole dataframe:")

count_result_df = count_result_df.sort_values(by=['config_id'])
print(count_result_df)


Total count: 930

Count per config_id for the 5 sub-dataframes and average count for the whole dataframe:
   language           config_id  df1  df2  df3  df4  df5  avg_df  df
3        fr  config_A_complex_1   10   10    9    9    9     9.4  47
0        fr  config_A_complex_2   10   10    9    9    9     9.4  47
1        fr     config_A_easy_1   10   10    9    9    9     9.4  47
2        fr     config_A_easy_2   10   10    9    9    9     9.4  47
4        fr   config_A_pseudo_1   10   10    9    9    9     9.4  47
5        fr   config_A_pseudo_2   10   10    9    9    9     9.4  47
9        fr  config_B_complex_1   11   11   11   11   11    11.0  55
10       fr  config_B_complex_2   11   11   11   11   11    11.0  55
11       fr     config_B_easy_1   11   11   11   11   11    11.0  55
8        fr     config_B_easy_2   11   11   11   11   11    11.0  55
6        fr   config_B_pseudo_1   11   11   11   11   11    11.0  55
7        fr   config_B_pseudo_2   11   11   11   11   11    11.0  

Values in cell above and below matches if we remove all rows with 'duplicate_trials' in the 'notes' column

In [14]:
# Initialize DataFrame to store unique language and config_id combinations
participant_result_df = pd.DataFrame(df_1[['language', 'config_id']].drop_duplicates())

# Process each DataFrame and add the counts to the result DataFrame
for i, dfi in enumerate([df_1, df_2, df_3, df_4, df_5], start=1):
    # Count unique participants per language and config_id
    participant_counts = dfi.groupby(['language', 'config_id'])['participant_id'].nunique().reset_index()
    participant_counts.rename(columns={'participant_id': f'df{i}'}, inplace=True)
    participant_result_df = pd.merge(participant_result_df, participant_counts, on=['language', 'config_id'], how='left')

# Calculate the average number of participants per (language, config_id) for the whole dataset
average_participants = df.groupby(['language', 'config_id'])['participant_id'].nunique().div(5).reset_index()
average_participants['participant_id'] = average_participants['participant_id'].round(2)
average_participants.rename(columns={'participant_id': 'avg_df'}, inplace=True)
participant_result_df = pd.merge(participant_result_df, average_participants, on=['language', 'config_id'], how='left')

# Calculate the total number of participants per (language, config_id) for the whole dataset
total_participants = df.groupby(['language', 'config_id'])['participant_id'].nunique().reset_index()
total_participants.rename(columns={'participant_id': 'df'}, inplace=True)
participant_result_df = pd.merge(participant_result_df, total_participants, on=['language', 'config_id'], how='left')

# Round the participant counts to the nearest integer
participant_result_df.iloc[:, 2:] = participant_result_df.iloc[:, 2:].round(decimals=0).astype(int)

# Print the resulting DataFrame
print("Total number of participants:", len(df['participant_id'].unique()))

for phase in df['phase'].unique():
    for lan in df['language'].unique():
        print("\nTotal number of unique participants for each phase in", lan)
        filtered_df = df[(df['language'] == lan) & (df['phase'].apply(lambda x: x.split("_")[0] in [phase]))]
        print(f"{phase}: {len(filtered_df['participant_id'].unique())} participants")

print("\nTotal number of unique participants per config_id across all dataframes and average participants:")
participant_result_df = participant_result_df.sort_values(by=['config_id'])
print(participant_result_df)   

# only print config_id and df columns
print(participant_result_df[['config_id', 'df']])


Total number of participants: 155

Total number of unique participants for each phase in fr
PSEUDO: 155 participants

Total number of unique participants for each phase in fr
WORD: 155 participants

Total number of unique participants per config_id across all dataframes and average participants:
   language           config_id  df1  df2  df3  df4  df5  avg_df  df
3        fr  config_A_complex_1   10   10    9    9    9     9.0  47
0        fr  config_A_complex_2   10   10    9    9    9     9.0  47
1        fr     config_A_easy_1   10   10    9    9    9     9.0  47
2        fr     config_A_easy_2   10   10    9    9    9     9.0  47
4        fr   config_A_pseudo_1   10   10    9    9    9     9.0  47
5        fr   config_A_pseudo_2   10   10    9    9    9     9.0  47
9        fr  config_B_complex_1   11   11   11   11   11    11.0  55
10       fr  config_B_complex_2   11   11   11   11   11    11.0  55
11       fr     config_B_easy_1   11   11   11   11   11    11.0  55
8        fr  

In [17]:
# type of accuracy column of first row
print("Type of accuracy column of first row: ", type(df.loc[0]['accuracy']))

Type of accuracy column of first row:  <class 'str'>
